<a href="https://colab.research.google.com/github/tiff03/Plant-Growth-and-Water-Usage-Data-Analysis/blob/main/I_Simulated_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulated Data
---
plant name: length_I, length_D, length_M, length_L


tomato: 15, 30, 28, 17



In [32]:
import numpy as np
from matplotlib import pyplot as plt
import time
from time import time, ctime
from time import struct_time
import datetime
import random
import pandas as pd
from tabulate import tabulate
import csv
from csv import writer
from google.colab import drive

path = '/content/drive/My Drive/SUMMER 22/intern/I_data/I_simulated_data.csv'


tomato_stage_data = [15, 30, 28, 17]
head = ["time stamp", "event", "sensor ID", "water reserve (mm)", "volume (mm)"]

Stage I (Initial)

In [34]:
minute_interval = 5 #how many minutes between each sensor read
sensor_ID = "1"
water_reserve = 10 #how much water initially in the reserve
days_since_water = 2 #days since plant has been watered - set at 2 so it waters first day
irrigation_volume = 0 
IR = 0
watered = False
water_time = 9 #what time to water the plants at
all_data = []
IR_data_point = []


today = datetime.datetime(2022, 5, 19)

end_time = today

#for stage in range(len(tomato_stage_data)):   account for all stages
  #for t in range(tomato_stage_data[stage]):

#elapse time every 5 minutes
for t in range(tomato_stage_data[0]):       #only I stage right now
  
  watered = False

  for h in range(24):
    time_now = end_time.time() 

    if(time_now.hour % 6 == 0):
      water_reserve -= 0.25

    for m in range(60//minute_interval):
      end_time += datetime.timedelta(minutes = minute_interval)

      if(not watered and (time_now.hour == water_time and time_now.minute == 0)):
        irrigation_volume = random.randint(1,3)
        water_reserve += irrigation_volume
        watered = True
        IR_data_point = [time_now.strftime("%m/%d/%Y %H:%M:%S"), "IR", None, None, irrigation_volume]
        all_data.append(IR_data_point)

      
      data_point = [end_time.strftime("%m/%d/%Y %H:%M:%S"), "sensor", sensor_ID, water_reserve, None]
      all_data.append(data_point)
      
      irrigation_volume = 0
      

with open(path, 'w') as file:
  writer = csv.writer(file)

  writer.writerow(head)
  
  for point in all_data:
    writer.writerow(point)

In [41]:
dataset = pd.read_csv(path)

dataset["sensor ID"] = dataset["sensor ID"].astype(object)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4335 entries, 0 to 4334
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   time stamp          4335 non-null   object 
 1   event               4335 non-null   object 
 2   sensor ID           4320 non-null   object 
 3   water reserve (mm)  4320 non-null   float64
 4   volume (mm)         15 non-null     float64
dtypes: float64(2), object(3)
memory usage: 169.5+ KB


In [36]:
numeric_columns = list(dataset.select_dtypes(include=['float64']).columns)
def_columns = list(dataset.select_dtypes(include=['object']).columns)

print('Numerical Columns: ',numeric_columns)
print('Defining Columns: ',def_columns)

dataset.describe()

Numerical Columns:  ['water reserve (mm)', 'volume (mm)']
Defining Columns:  ['time stamp', 'event', 'sensor ID']


,water reserve (mm),volume (mm)
count,4320.000000,15.000000
mean,18.308333,2.133333
std,5.531494,0.833809
min,9.500000,1.000000
25%,13.937500,1.500000
50%,18.000000,2.000000
75%,22.312500,3.000000
max,27.500000,3.000000


Graph for Stage I


---


usage per 5 minutes

In [37]:
time_x = []
usage_y = []
previous_point_index = None
irrigation = 0
x = 0

for i in range(len(dataset['time stamp'])):
  if(dataset['event'][i] == 'IR'):
    irrigation = dataset['volume (mm)'][i]

  if(dataset['water reserve (mm)'][i] != None and previous_point_index != None): 
    usage_y.append(dataset['water reserve (mm)'][i] - dataset['water reserve (mm)'][previous_point_index] - irrigation)
    time_x.append(dataset['time stamp'][i])
    irrigation = 0
    x += 1
    
  previous_point_index = i
  
x = [i for i in range(x)]

plt.figure(figsize=(150,100))
plt.title("Water Usage over Time for Stage I", fontsize=300)
plt.xlabel('Time', fontsize=100)
plt.ylabel('Water Usage (mm)', fontsize=100)

plt.tick_params(axis='x', labelsize=80)
plt.tick_params(axis='y', labelsize=80)

default_x_ticks = range(len(x))
plt.xticks(default_x_ticks, time_x, fontsize = 80)
plt.scatter(default_x_ticks, usage_y, s = 1000)


Graph for Stage I


---
usage per day



In [44]:
from dateutil import parser

time_x = 0
usage_y = []
previous_point_index = None
irrigation = 0
water_reserve_initial = 10
interval = 24
x = 0

for i in range(len(dataset['time stamp'])):

  date = datetime.datetime.strptime(dataset['time stamp'][i], '%m/%d/%Y %H:%M:%S')

  if(dataset['event'][i] == 'IR'):
    irrigation = dataset['volume (mm)'][i]

  if(date.hour % interval == 0 and date.minute == 0): 
    usage_y.append(dataset['water reserve (mm)'][i] - dataset['water reserve (mm)'][previous_point_index] - irrigation)
    time_x+=1
    irrigation = 0
    x += 1
    
  previous_point_index = i
  

plt.figure(figsize=(150,100))
plt.title("Water Usage over Time for Stage I", fontsize=300)
plt.xlabel('Time', fontsize=100)
plt.ylabel('Water Usage (mm)', fontsize=100)

plt.tick_params(axis='x', labelsize=80)
plt.tick_params(axis='y', labelsize=80)

default_x_ticks = range(time_x)
plt.xticks(default_x_ticks, range(1, time_x+1), fontsize = 80)
plt.scatter(default_x_ticks, usage_y, s = 1000)

Stage D (Development)